## Install requerements

In [ ]:
#torch windows cuda (nvidia)
!pip install requirements.txt
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install tensorflow[and-cuda]

In [ ]:
import os
from datetime import datetime
import time
import pyaudio
import librosa
import asyncio
import threading
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, AutoProcessor, AutoModelForCTC, TFWav2Vec2ForCTC, \
                         AutoTokenizer, AutoFeatureExtractor, T5ForConditionalGeneration, T5Tokenizer
import spacy
from spellchecker import SpellChecker

import torch
import torchaudio
import tensorflow

In [2]:
#TODO когда санкции снимут
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./google_cred.json"

# auto-translator
Program for auto translate any video in real time

## ПРЕДИСЛОВИЕ

#### Перед тем, как запускать прогу, необходимо:

1) установить virtual audio cable

2) в sound mixer options для всех выбрать defaul input = Line 1(virtual audio cable) ( можете выбрать любой удобный вам кабель)

3) затем для нужной программы (окна) в output выбрать тот же кабель (Line 1, например) 

4) вызвать код для нахождения подключённых аудиоустройств:

   ```recorder = pyaudio.PyAudio()```

   ```for i in range(recorder.get_device_count()):```
   
   ```    print(i, recorder.get_device_info_by_index(i))```

5) выбрать подходящее аудиоустройство (для записи должно быть устройство Line 1, у которого ```'maxInputChannels' != 0```,
  а для вывода должно быть устройство (наушники, колонки), у которого ```'maxOutputChannels' != 0```

6) и когда открываете потоки для записи/воспроизведения звука в kwargs передаёте нужный вам индекс в переменную ```input_device_index```, аналогично для ```input_device_index``` в воспроизводящем потоке

7) после запуска у нас звук будет перенаправляться так: приложение->Line_1_output->Line_1_input->Наша прога->Вывод на выбранное устройство.


In [3]:
class AudioChanel():
    def __init__(self):
        #change this according to your model
        self.chunk = 1024
        self.format = pyaudio.paInt16
        self.channels = 1#2
        self.rate = 16000
        self.recorder = pyaudio.PyAudio()
        self.frames = []
        self.seconds = 2
        # Открыть поток для записи
        #1 {'index': 1, 'structVersion': 2, 'name': 'Line 1 (Virtual Audio Cable)', 'hostApi': 0, 'maxInputChannels': 2, 
        #   'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 
        #   'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
        self.input_stream = self.recorder.open(format=self.format,
                                          channels=self.channels,
                                          rate=self.rate,
                                          input=True,
                                          frames_per_buffer=self.chunk,
                                          input_device_index=1) #0,1 - work
        # Открыть поток для воспроизведения
        #5 {'index': 5, 'structVersion': 2, 'name': '... (Realtek)', 'hostApi': 0, 'maxInputChannels': 0, 
        #   'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 
        #   'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
        self.output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        
        
    def record(self):
        while True:
            data = self.input_stream.read(self.chunk)
            self.frames.append(data)

    def play(self, audio = None):
        output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        match audio:
            case list():
                for frame in audio:
                    output_stream.write(frame)
            case _:
                output_stream.write(audio.tobytes())

    def play_default(self):
        output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        with threading.Lock():
            to_play = list(self.frames)
        for frame in to_play:
            output_stream.write(frame)
    
    def get(self):
        with threading.Lock():
            return list(self.frames)
        
        
    def close(self):
        self.input_stream.stop_stream()
        self.input_stream.close()

In [ ]:
class Agent():
    def __init__(self):
        self.audio = AudioChanel()
        self.device = torch.device('cuda')
        #you may use your models instead
        self.processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
        self.model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(self.device)
        self.feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

        self.spell = SpellChecker()
        self.nlp = spacy.load('en_core_web_sm')#en_core_web_trf
        self.PASS_VAR = False

        #the worst model EVER
        model_name = 'utrobinmv/t5_translate_en_ru_zh_small_1024'# small, large and base
        self.model_translate = T5ForConditionalGeneration.from_pretrained(model_name).to(self.device)
        self.tokenizer_translate = T5Tokenizer.from_pretrained(model_name)
        
        language = 'ru'
        model_id = 'v4_ru'
        self.sample_rate = 48000
        self.speaker = 'kseniya'
        
        self.model_voice, _ = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                             model='silero_tts',
                                             language=language,
                                             speaker=model_id)
        self.model_voice.to(self.device)
        self.to_play = []
    
    def correcting(self,sent):
        to_spell = sent.split()
        corrected_words = [self.spell.correction(word) for word in to_spell]
        return ' '.join((word if word is not None else to_spell[idx]) for idx, word in enumerate(corrected_words))

    def send_to_play(self):
        while True:
            time.sleep(1)
            with threading.Lock():
                copy_to_play = list(self.to_play)
            while len(copy_to_play):
                to_play = copy_to_play[0]
                translation = threading.Thread(target=self.audio.play,args=(to_play[0],))
                original = threading.Thread(target=self.audio.play,args=(to_play[1],))
                
                translation.start()
                original.start()
                
                translation.join()
                original.join()
                with threading.Lock():
                    del self.to_play[0]
                    del copy_to_play[0]
    
    async def run(self):
        #start recording
        _ = threading.Thread(target=self.audio.record).start()
        _ = threading.Thread(target=self.send_to_play).start()
        
        # await asyncio.sleep(10)
        last_state = ''
        while True:
            '''
            PART 1
            RECORDING
            description: record every 2 seconds of audio from PC
            '''
            await asyncio.sleep(2)
            time = datetime.now()
            frames = self.audio.get()[:200]
            to_model = np.frombuffer(b''.join(frames), dtype=np.int16)
            end_1 = datetime.now()
            # print('end_1: ', end_1)
            '''
            PART 2
            CONVERTING AUDIO TO TEXT
            description: put audio into model and get recognized words
            '''
            input_values = self.processor(to_model, sampling_rate = self.audio.rate, return_tensors="pt").input_values  # Batch size 1
            logits = self.model(input_values.to(torch.float).to(self.device)).logits
            predicted_ids = torch.argmax(logits, axis=-1)
            
            outputs = self.tokenizer.decode(predicted_ids[0], output_word_offsets=True)
            # compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
            time_offset = self.model.config.inputs_to_logits_ratio / self.feature_extractor.sampling_rate
            
            word_offsets = [
                {
                    "word": d["word"],
                    "start_time": round(d["start_offset"] * time_offset, 2),
                    "end_time": round(d["end_offset"] * time_offset, 2),
                }
                for d in outputs.word_offsets
            ]
            end_2 = datetime.now()
            print('end_2: ', end_2-end_1)
            '''
            PART 3
            FIND SENTENCIES IN THE OUTPUT TEXT
            description: find grammar parts in output text for building whole sentences
            '''
            text = ' '.join([word['word'] for word in word_offsets])
            fixed = self.correcting(text)
            print('Full recorded text: ', text)
            if last_state == '':
                last_state = text
            else:
                if last_state == text:
                    self.PASS_VAR = True
                last_state = text
                
            doc = self.nlp(fixed)
    
            end_3 = datetime.now()
            print('end_3: ', end_3-end_2)
            '''
            PART 4
            TRANSLATE SENTENCE INTO RUSSIAN (or any other language)
            description: translate one of the list, because we should be sure about full context
            '''
            sents = list(doc.sents)
            if len(sents)<=1 and not self.PASS_VAR:
                continue
            self.PASS_VAR = False
            print(sents)
            
            prefix = 'translate to ru: '
            ### delete part of audio.frames list
            with threading.Lock():
                idx_del = len(sents[0].text.split())-1 if len(sents[0].text.split())==len(word_offsets) else len(sents[0].text.split())
                to_del = int(word_offsets[idx_del]['end_time']*self.audio.rate/self.audio.chunk)
                play_background = self.audio.frames[:to_del+1]
                del self.audio.frames[:to_del]
            ###
            src_text = prefix + sents[0].text
            
            # translate Englosh to Russian
            input_ids = self.tokenizer_translate(src_text, return_tensors="pt")
            
            generated_tokens = self.model_translate.generate(**input_ids.to(self.device))
            
            result = self.tokenizer_translate.batch_decode(generated_tokens, skip_special_tokens=True)
            end_4 = datetime.now()
            print('end_4: ', end_4-end_3)
            '''
            PART 5
            VOICEOVER OF TEXT AND PLAY
            description: voice translated text
            '''
            print(result[0])
            if not len(result[0]):
                continue
                  
            audio = self.model_voice.apply_tts(text=result[0],
                        speaker=self.speaker,
                        sample_rate=self.sample_rate)
            
            audio_numpy = audio.cpu().numpy()
            audio_resampled = librosa.resample(audio_numpy, orig_sr=self.sample_rate, 
                                               target_sr=self.audio.rate, res_typestr='fft')
            audio_int16 = (audio_resampled * 32767).astype(np.int16)
            end_5 = datetime.now()
            print('end_5: ', end_5-end_4)
            # self.audio.play(audio_int16)
            with threading.Lock():
                self.to_play.append((audio_int16,frames[:to_del]))
            # _ = threading.Thread(target=self.audio.play, args=(play_background,)).start()
            
a = Agent()
await a.run()